In [1]:
import yaml
from box import ConfigBox
from box.exceptions import BoxValueError
from pathlib import Path
import os
from pydantic import BaseModel
import urllib.request as request
from utils import *
import requests
import zipfile

In [2]:

class DataIngestionConfigValidator(BaseModel):
    root_dir: Path
    source_url: str
    zipped_local_dir: Path #Path #data_dir:
    unzip_dir: Path

def validate_data(configbox):
    config = configbox.data_ingestion
    config = DataIngestionConfigValidator(**config)
    return config

In [3]:
class DataIngestion:
    def __init__(self, config):
        self.config = config
        
    def download_file(self):
        """
        Url: str
        downloads data into a specified directory
        """

        zipped_local_dir = os.path.join(self.config.root_dir, "adult.zip")
        create_directories([self.config.root_dir])
        
        if not os.path.exists(self.config.zipped_local_dir):
            response = requests.get(self.config.source_url)
            #print(response.content)
            with open(zipped_local_dir, 'wb') as file:
                file.write(response.content)
        else:
            print(f"File already exists in {self.config.zipped_local_dir}")
        
        
    def unzip_file(self):
        """
            zip_file_path: str
            Extracts the zip file into the data directory
            Function returns None
            """
        target_dir = self.config.unzip_dir
        with zipfile.ZipFile(self.config.zipped_local_dir, 'r') as files:
            files.extractall(target_dir)

In [4]:
config_box = read_yaml_file(Path('artifacts/ConfigFiles/config.yaml'))
config = validate_data(config_box)
data = DataIngestion(config)
data.download_file()

yaml file: artifacts\ConfigFiles\config.yaml loaded successfully
created directory at artifacts\data_ingestion
File already exists in artifacts\data_ingestion\adult.zip


In [5]:
data.unzip_file()